In [13]:
class centralNode:
    def __init__(self,value=None,right=None,left=None,labelled=False,nextNode=None):
        self.value=value
        self.right=right
        self.left=left
        self.labelled=labelled
        self.nextNode=nextNode

In [14]:
class rightNode:
    def __init__(self,value=None,children=None,labelled=False):
        self.value=value
        self.children = children if children is not None else []
        self.labelled=labelled
    def assignChildren(self,p):
        for i in range(p-1):
            if i==0:
                firstOne=leafNode(i+103)
                self.children.append(firstOne)
            elif i==p-2:
                lastOne=leafNode(i+103)
                firstOne.nextNode=lastOne
                self.children.append(lastOne)
            else:
                self.children.append(leafNode(i+103))

In [15]:
class leftNode:
    def __init__(self,value=None,children=None,labelled=False):
        self.value=value
        self.children = children if children is not None else []
        self.labelled=labelled
    def assignChildren(self,p):
        for i in range(p-1):
            if i==0:
                firstOne=leafNode(i+103)
                self.children.append(firstOne)
            elif i==p-2:
                lastOne=leafNode(i+103)
                firstOne.nextNode=lastOne
                self.children.append(lastOne)
            else:
                self.children.append(leafNode(i+103))

In [16]:
class leafNode:
    def __init__(self,value=None,labelled=False,nextNode=None):
        self.value=value
        self.labelled=labelled
        self.nextNode=nextNode

In [17]:
def create_lobster(n,p,prev,root,i):
    if n==0:
        return 
    a=centralNode(i*100)
    b=rightNode(i*101)
    c=leftNode(i*102)
    a.right=b
    a.left=c
    b.assignChildren(p)
    c.assignChildren(p)
#     print(len(b.children))
#     print(len(c.children))
    if prev!=None:
        prev.nextNode=a
    else:
        root=a
    create_lobster(n-1,p,a,root,i+1)
    return root

In [18]:
import math
def generateVerticesEdges(n,p):
    possible_vertices=[i for i in range(1,math.ceil(((2*n*(p+1)+n)/2))+1)]
    possible_edges=[i for i in range(2,2*n*(p+1)+n+1)]
    return possible_vertices,possible_edges

In [19]:
def printNodes(root):
    print(root.__class__.__name__)
    if root.__class__.__name__=="centralNode":
        print(root.value,"-->",root.right.value,"Edge weight:",root.value+root.right.value,":",root.value,"-->",root.left.value,"Edge weight:",root.value+root.left.value)
        printNodes(root.right)
#         print(root.right.children)
#         print(root.left.children)
        printNodes(root.left)
        printNodes(root.nextNode)
    elif root.__class__.__name__=="leftNode" or root.__class__.__name__=="rightNode":
        children=root.children
        for i in children:
            print(root.value,"-->",i.value,"Edge weight:",root.value+i.value)
            if i.nextNode is not None:
                temp="Extra Edge "+str(i.value)+" --> "+str(i.nextNode.value)+" Edge weight:"+str(i.value+i.nextNode.value)
        print(temp)

In [20]:
import math
def labelNodes(root,possible_edges,possible_vertices):
    if root is None:
        return possible_edges,possible_vertices
    if root.__class__.__name__=="centralNode":
        if not root.labelled:
            root.value=1
            root.labelled=True
#         print("centralNode value",root.value)
        min_search=math.floor(min(possible_edges)/2)
        for i in range(min_search-1,len(possible_vertices)):
            if root.value+possible_vertices[i] in possible_edges:
                root.right.value=possible_vertices[i]
                root.right.labelled=True
#                 print(root.right.value)
                possible_edges.remove(root.value+possible_vertices[i])
#                 print(possible_edges)
                break
#         print(root.right.labelled)
        if min(possible_edges)==root.value+root.right.value+1:
#         print(possible_edges)
            possible_edges,possible_vertices=labelNodes(root.right,possible_edges,possible_vertices)
            min_search=math.floor(min(possible_edges)/2)
            for i in range(min_search-1,len(possible_vertices)):
                if root.value+possible_vertices[i] in possible_edges:
                    root.left.value=possible_vertices[i]
                    possible_edges.remove(root.value+root.left.value)
                    break
            x,y=generate_first_last(min(possible_edges),len(root.left.children),possible_vertices)
            root.left.children[0].value,root.left.children[-1].value=x,y
            root.left.children[0].labelled,root.left.children[-1].labelled=True,True
            possible_edges.remove(x+y)
            possible_edges.remove(root.left.value+x)
            possible_edges.remove(root.left.value+y)
    #         print(root.left.children[0].value,root.left.children[-1].value)
            possible_edges,possible_vertices=labelNodes(root.left,possible_edges,possible_vertices)
            if root.nextNode:
                root.nextNode.value=min(possible_edges)-root.value
                root.nextNode.labelled=True
                possible_edges.pop(0)
#                 print("Debug",possible_edges)
                possible_edges,possible_vertices=labelNodes(root.nextNode,possible_edges,possible_vertices)
        else:
            possible_edges=assignChildrenv2(root,possible_edges)
            if root.nextNode:
                root.nextNode.value=min(possible_edges)-root.value
                root.nextNode.labelled=True
                possible_edges.pop(0)
#                 print("Debug",possible_edges)
                possible_edges,possible_vertices=labelNodes(root.nextNode,possible_edges,possible_vertices)
        return possible_edges,possible_vertices
    elif root.__class__.__name__=="rightNode":
#         print(root.__class__.__name__)
#         print(possible_edges,root.value)
        flag=None
        firstValue=None
        for child in root.children:
            minimum=min(possible_edges)
            min_search=math.floor(minimum/2)
            for i in range(min_search-1,minimum+1):
                if root.value + possible_vertices[i] in possible_edges:
                    child.value=possible_vertices[i]
                    possible_edges.pop(0)
                    break
            if flag is None:
                flag=True
                firstValue=child.value
#                 print("firstValue",firstValue)
            lastValue=child.value
        possible_edges.remove(firstValue+lastValue)
        return possible_edges,possible_vertices
    elif root.__class__.__name__=="leftNode":
        for child in root.children:
            if child.labelled:
                continue
            else:
                for i in range(len(possible_vertices)):
                    if root.value+possible_vertices[i] in possible_edges:
                        child.value=possible_vertices[i]
                        break
                possible_edges.remove(root.value+possible_vertices[i])
        return possible_edges,possible_vertices

In [21]:
def generate_first_last(min_edge,length,possible_vertices):
    for i in range(len(possible_vertices)):
        if possible_vertices[i]==int((min_edge-length+2)/2):
            return possible_vertices[i],possible_vertices[i]+p-2

In [22]:
def assignChildrenv2(root,possible_edges):
#     print(root.right.value)
    for child in root.right.children:
        min_search=math.floor(min(possible_edges)/2)
        for i in range(min_search-1,len(possible_vertices)):
#             print(root.right.value+possible_vertices[i])
            if root.right.value+possible_vertices[i] in possible_edges:
                child.value=possible_vertices[i]
                possible_edges.remove(root.right.value+possible_vertices[i])
                break
#     print(root.right.children[0].value,root.right.children[-1].value)
    possible_edges.remove(root.right.children[0].value+root.right.children[-1].value)
#     print(possible_edges)
#     print(min(possible_edges),len(root.right.children),root.value)
    root.left.value=min(possible_edges)+len(root.right.children)-root.value+1
    possible_edges.remove(root.left.value+root.value)
    possible_edges.pop(0)
    for child in root.left.children:
        min_search=math.floor(min(possible_edges)/2)-len(root.right.children)+2
#         print(min_search)
        for i in range(min_search-1,len(possible_vertices)):
#             print(root.right.value+possible_vertices[i])
            if root.left.value+possible_vertices[i] in possible_edges:
                child.value=possible_vertices[i]
                possible_edges.remove(root.left.value+possible_vertices[i])
                break
    return possible_edges
        
        

In [23]:
import time
n=int(input("Enter n"))
p=int(input("Enter p"))
if p<3:
    print("Sorry code only works for p>2")
else:
    root=create_lobster(n,p,None,None,1)
    possible_vertices,possible_edges=generateVerticesEdges(n,p)
#     print(possible_edges)
#     start_time=time.time()
    labelNodes(root,possible_edges,possible_vertices)
#     print(time.time()-start_time)
    printNodes(root)

Enter n5
Enter p5
centralNode
1 --> 1 Edge weight: 2 : 1 --> 7 Edge weight: 8
rightNode
1 --> 2 Edge weight: 3
1 --> 3 Edge weight: 4
1 --> 4 Edge weight: 5
1 --> 5 Edge weight: 6
Extra Edge 2 --> 5 Edge weight:7
leftNode
7 --> 3 Edge weight: 10
7 --> 4 Edge weight: 11
7 --> 5 Edge weight: 12
7 --> 6 Edge weight: 13
Extra Edge 3 --> 6 Edge weight:9
centralNode
13 --> 7 Edge weight: 20 : 13 --> 13 Edge weight: 26
rightNode
7 --> 8 Edge weight: 15
7 --> 9 Edge weight: 16
7 --> 10 Edge weight: 17
7 --> 11 Edge weight: 18
Extra Edge 8 --> 11 Edge weight:19
leftNode
13 --> 9 Edge weight: 22
13 --> 10 Edge weight: 23
13 --> 11 Edge weight: 24
13 --> 12 Edge weight: 25
Extra Edge 9 --> 12 Edge weight:21
centralNode
14 --> 14 Edge weight: 28 : 14 --> 20 Edge weight: 34
rightNode
14 --> 15 Edge weight: 29
14 --> 16 Edge weight: 30
14 --> 17 Edge weight: 31
14 --> 18 Edge weight: 32
Extra Edge 15 --> 18 Edge weight:33
leftNode
20 --> 16 Edge weight: 36
20 --> 17 Edge weight: 37
20 --> 18 Edge we